In [1]:
# 修改checkpoint路径
ckpt_dir = '/home/hanlv/workspace/code/research/infodemic/LLM/swift/examples/pytorch/llm/output/openchat_3.5/with_solar_info/brave/data1-split=8:2-ratio=1.0/lr=1.1e-4-20240116-00:52:51/checkpoint-609'

import os
import sys
import json
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

dirs = [".."]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)

from swift.llm import (
    get_model_tokenizer, get_template, inference
)
from swift.tuners import Swift
from custom import CustomModelType, CustomTemplateType
import evaluation

with open(f"{ckpt_dir}/sft_args.json", "r") as f:
    sft_args = json.load(f)
model_type, template_type = sft_args["model_type"], sft_args["template_type"]
model, tokenizer = get_model_tokenizer(model_type, model_kwargs={'device_map': 'auto'})
model = Swift.from_pretrained(model, ckpt_dir, inference_mode=True)
model.generation_config.max_new_tokens = 512
model.generation_config.do_sample = False

template = get_template(template_type, tokenizer)

get_response = lambda prompt : inference(model, template, prompt)


2024-01-20 06:24:25,344 - modelscope - INFO - PyTorch version 2.1.2 Found.
2024-01-20 06:24:25,347 - modelscope - INFO - Loading ast index from /home/hanlv/.cache/modelscope/ast_indexer
2024-01-20 06:24:25,381 - modelscope - INFO - Loading done! Current index file version is 1.11.0, with md5 dc3cda2b56a86b0242cd145399a14071 and a total number of 953 components indexed
[INFO:swift] Setting torch_dtype: torch.bfloat16
[INFO:swift] model_config: MistralConfig {
  "_name_or_path": "/home/css/models/openchat-3.5-0106",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": fa

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
evaluation.cal_metric_single_llm(sft_args, get_response, save=True)


2199 / 2439
{0: 1817, 2: 381, -2: 0, 'wrong': 13}
ACC: 0.9940855323020928
F1: 0.989670436531965
Precision: 0.9891625750963848
Recall: 0.9901800706386428



KeyboardInterrupt: 